# Setup

In [57]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango

Note: you may need to restart the kernel to use updated packages.


In [58]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

In [59]:
# set up assistant

from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# Open the file and read the contents
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# print(prompt)

my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
)

In [60]:


# create a thread
my_thread = client.beta.threads.create()

import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown (with no delimiters) and with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


# Initial File Upload and Prompt

In [61]:
# get the job description from GitHub
import time


file_path = 'temp_files/jobdescription.txt'
# infra engineer example
#!wget -O file_path https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# medical assistant example
!wget -O 'temp_files/jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

# infra engineer example
# !wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# medical assistant example
!wget -O 'temp_files/companyvalues.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
# time.sleep(5)


# Upload the file
file_company_values = client.files.create(
  file=open("temp_files/companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("temp_files/jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-20 14:23:55--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483 (2.4K) [text/plain]
Saving to: ‘temp_files/jobdescription.txt’

temp_files/jobdescr 100%[===================>]   2.42K  --.-KB/s    in 0s      

2023-12-20 14:23:55 (6.65 MB/s) - ‘temp_files/jobdescription.txt’ saved [2483/2483]

--2023-12-20 14:23:56--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/about-company.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

# Priming the Pump

In [62]:
# Refresh the context to ensure it has read the files
primer_values = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the company values file, using the company name.
            """, [file_job_description.id, file_company_values.id])
print(primer_values)

## Values of Stanford Health Care - University Healthcare Alliance

- **Innovation:** Embracing innovation, change, and continuous improvement to raise the quality of care and shape the future of medicine.
  
- **Connection:** Cultivating community, connection, and care for each other to nurture trust, respect, belonging, and shared purpose.
  
- **Teamwork:** Collaborating across roles, ranks, units, and departments to unlock possibilities for greater impact and fulfillment.
  
- **Mentorship:** Nurturing and pursuing ongoing professional growth to enjoy impactful and fulfilling careers.
  
- **Participation:** Freely sharing opinions and valuing diverse backgrounds and contributions to improve decision-making, outcomes, and engagement.
  
- **Action:** Uniting diverse perspectives and expertise for the greater good.




In [63]:
# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run_and_display_response(client,my_thread,my_assistant,"""
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id, file_company_values.id])
print(primer_job)


### Job Title

Medical Assistant II

### Technical Competencies

#### 1. Clinical Procedures
Knowledge and ability to perform standard clinical tasks and procedures with precision and adherence to best practices.
- **Patient Preparation:** Assisting in preparing patients for examination or procedures, ensuring patient comfort and adherence to privacy regulations.
- **Procedure Assistance:** Providing direct support during various medical procedures, understanding the requirements and steps involved.
- **Equipment Operation:** Skilled use of clinical equipment, ensuring proper function, maintenance, and sterilization.
- **C-I-CARE Protocol Compliance:** Applying the C-I-CARE framework to deliver compassionate and efficient patient care.

#### 2. Medical Knowledge
An advanced understanding of medical terminology, patient care techniques, and healthcare policies and procedures.
- **Terminology Proficiency:** Accurate use and understanding of medical terms and abbreviations in verbal and w

# Create Introduction

In [64]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"""
            Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.
            """)
print(introduction)

# Interview Playbook for Medical Assistant II

## Introduction

This Interview Playbook serves as a comprehensive guide to evaluate a Medical Assistant II's candidacy at Stanford Health Care - University Healthcare Alliance. It includes a focused approach to examining both the technical skills and behavioral characteristics of potential candidates, ensuring alignment with the role's responsibilities and the organization's core values: Innovation, Connection, Teamwork, Mentorship, Participation, and Action. The playbook aims to provide a structured interview process that highlights a candidate's competencies in clinical procedures, medical knowledge, EMR proficiency, communication, and organizational skills, along with their behavioral attributes like patient-centric focus, team collaboration, continuous learning, commitment to JEDI (Justice, Equality, Diversity, and Inclusion), and an initiative-driven approach to outcomes.

## Interview Structure

The interview for the Medical Assista

# Create Evaluation Criteria

In [65]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

## Evaluation Criteria

During the interview process, candidates for the Medical Assistant II position will be evaluated against a set of predefined competencies that align with the responsibilities of the role and the core values of Stanford Health Care - University Healthcare Alliance. The following job scorecard includes sections for technical and behavioral competencies, as well as areas for practical assignment and team interaction assessment, if applicable.

| Competency                            | Assessment | Comments |
|---------------------------------------|------------|----------|
| Technical: Clinical Procedures        |            |          |
| Technical: Medical Knowledge          |            |          |
| Technical: EMR Proficiency            |            |          |
| Technical: Communication Skills       |            |          |
| Technical: Multi-tasking and Organizational Skills |            |          |
| Behavioral: Patient-Centric Focus     |            |  

# Create Technical Assessment

In [66]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

Evaluating a candidate's ability to perform clinical tasks and utilize medical knowledge is crucial in identifying a skilled Medical Assistant II. By assessing these technical abilities, we aim to ensure that the candidate possesses the qualifications necessary for quality patient care and efficient clinic operations, aligning with the core value of Innovation at Stanford Health Care - University Healthcare Alliance.

### Technical Competence 1: Clinical Procedures

_Clinical Procedures refer to the candidate's hands-on skills and understanding of medical protocols necessary to support physicians and provide patient care effectively._

#### Lead Question

"Describe a time when you had to adapt a standard clinical procedure due to unexpected circumstances. How did you ensure patient safety and care quality while adjusting the procedure?"

_Look for an answer that demonstrates the candidate's ability to think on their feet, maintain standards under pressure, and 

In [67]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: Medical Knowledge

_Medical knowledge is critical for a Medical Assistant II to provide high-quality care, ensuring they understand various medical conditions, potential treatments, and healthcare procedures._

#### Lead Question

"Please explain how you have applied your medical knowledge to a real-world situation to improve a patient's health outcome."

_Look for specificity in the response that reflects the candidate's ability to translate theoretical knowledge into practical patient care._

#### Probing Questions

- "How do you stay current with medical terminology and do you have a method for learning new terms and concepts?"
  _Assess the candidate's commitment to continual learning and staying updated with the evolving medical field._

- "Describe the process you follow to ensure policy compliance within the clinic. Can you give an example of how you have handled a policy change?"
  _Check for the candidate's ability to adapt to and implement new poli

In [68]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Electronic Medical Records (EMR) Proficiency

_EMR Proficiency involves the candidate's skilled use and management of digital patient records, ensuring they handle sensitive information with attention to data accuracy and patient privacy._

#### Lead Question

"Tell us about a time when you recognized a discrepancy in a patient’s electronic medical record. How did you go about addressing and correcting it?"

_Expect descriptions of their attention to detail, respect for patient privacy, and actions to rectify errors in EMR._

#### Probing Questions

- "What is your process for ensuring the accuracy of data when entering patient information into the EMR system?"
  _Check their meticulousness and strategies for data verification._

- "Describe how you navigate between different sections in an EMR software when updating or retrieving patient information."
  _Assess their familiarity and efficiency with EMR software, reflecting their technical aptitude._

- "How

In [69]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Communication Skills

_Communication skills are vital for effectively exchanging information with patients, team members, and healthcare professionals, ensuring clarity and compassion in every interaction._

#### Lead Question

"Provide an example of a complex clinical situation you communicated to a patient or their family members. How did you ensure that they understood the information and the next steps?"

_The interviewer should look for answers that demonstrate empathetic and clear communication, tailored to the listener's level of understanding._

#### Probing Questions

- "How do you approach explaining medical procedures or care plans to patients who may be anxious or have difficulty understanding medical jargon?"
  _Gauge the candidate's ability to simplify complex information and provide reassurance._

- "Describe a scenario where you had to coordinate patient care with other healthcare professionals. What communication strategies did you use?"
  _

In [70]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Multi-tasking and Organizational Skills

_Multi-tasking and Organizational Skills are crucial for managing the varied and fast-paced demands of a clinical environment, ensuring efficiency and effective patient care._

#### Lead Question

"Describe a day where you had to juggle multiple tasks with tight deadlines. How did you prioritize and what was the outcome?"

_The interviewer should look for the candidate's ability to effectively prioritize tasks, manage their time, and adapt to changing situations without compromising quality of care._

#### Probing Questions

- "How do you determine which patient needs or clinical tasks to address first when faced with simultaneous demands?"
  _Gauge the candidate's judgment in prioritizing tasks based on urgency and importance._

- "Can you provide an example of how you've streamlined your workflow in the clinic to improve efficiency?"
  _Assess their innovative approaches to optimizing workflows and enhancing patient

In [71]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [72]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

The aim is to delve into a candidate's adherence to values such as empathy and inclusivity, ensuring their alignment with the culture and ethos at Stanford Health Care - University Healthcare Alliance.

### Behavioral Competence 1: Patient-Centric Focus

_A Patient-Centric Focus entails prioritizing the well-being and satisfaction of patients, embodying the Connection value, and contributing to a supportive healthcare environment._

#### Lead Question

"Can you share an experience where you went above and beyond for a patient? What drove you to take those extra steps?"

_Look for indications of genuine empathy, a dedication to patient care, and a reflection of the company's value of Connection._

#### Probing Questions

- "How do you ensure that each patient feels heard and cared for during their visit?"
  _Evaluate their ability to demonstrate empathy and attentiveness to individual patient needs._

- "Can you tell me about a time when you had to quickly resp

In [73]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Team Collaboration

_Team Collaboration involves working alongside colleagues to achieve common goals, reflecting the Teamwork value, and enhancing team dynamics and patient outcomes._

#### Lead Question

"Can you describe a situation where you had to collaborate with a team to address a challenge in the clinic? What was your role, and what was the outcome?"

_The interviewer should look for clear examples of cooperation, contribution to team efforts, and outcomes that reflect the ability to work harmoniously with others._

#### Probing Questions

- "How do you approach working with other departments to ensure comprehensive patient care?"
  _Gauge the candidate's experience with and commitment to cross-functional collaboration._

- "Describe how you resolve differences or conflicts within your team. Can you give a specific example?"
  _Assess their conflict resolution skills and their approach to maintaining a positive work environment._

- "Tell us about 

In [74]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Continuous Learning

_Continuous Learning is about the ongoing pursuit of knowledge and skills relevant to the medical field, signifying a commitment to personal growth and the Mentorship value of Stanford Health Care - University Healthcare Alliance._

#### Lead Question

"Tell us about a time when you identified a gap in your professional skills or knowledge and took steps to address it. What was the outcome?"

_The interviewer should look for examples that show the candidate's proactive approach to learning and willingness to improve._

#### Probing Questions

- "How do you keep your clinical skills up to date with the latest best practices?"
  _Assess their dedication to staying abreast of medical advancements and their strategies for continuous learning._

- "Can you describe a situation where you received feedback that led to a significant improvement in your work?"
  _Gauge their receptiveness to feedback and their ability to positively integrate it 

In [75]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Justice, Equality, Diversity, and Inclusion (JEDI)

_JEDI centers on fostering an inclusive healthcare environment that values diverse perspectives, aligning with the Participation value, and enhancing decision-making and patient care._

#### Lead Question

"Can you share an experience where you actively promoted or supported diversity and inclusion within your workplace? What motivated you, and what was the impact?"

_The interviewer should assess the candidate’s motivation for promoting JEDI principles and the tangible impact of their actions._

#### Probing Questions

- "How do you demonstrate cultural competence in your interactions with patients from diverse backgrounds?"
  _Gauge the candidate’s awareness of cultural sensitivities and their approach to inclusive care._

- "Can you describe a time when inclusive communication was critical in your role? What approach did you take?"
  _Evaluate their ability to communicate in a manner that includes contr

In [76]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Initiative and Outcome-Driven Performance

_Initiative and Outcome-Driven Performance assess the candidate's proactive approach to improving processes and systems, signifying alignment with the company's value of Action, and commitment to excellent patient care._

#### Lead Question

"Describe a project or initiative you started that had a significant positive impact on patient outcomes or clinic operations. What motivated you to start it, and what was the result?"

_The interviewer should look for evidence of proactive behavior, results-oriented actions, and the candidate’s capacity to identify and execute improvements._

#### Probing Questions

- "When faced with a problem in the clinic, how do you go about solving it? Can you give us a specific example?"
  _Assess the candidate’s problem-solving skills and their inclination to take initiative when issues arise._

- "How have you contributed to quality improvement initiatives in the past?"
  _Determine th

In [77]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [78]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment for the Medical Assistant II position at Stanford Health Care - University Healthcare Alliance is designed to assess candidates’ real-world application of their clinical and administrative skills in a controlled environment. This assignment will provide insight into their competency, adaptability, and ability to apply knowledge to practical challenges.

### Description

The candidate will be given a simulated clinic environment where they are required to complete a series of tasks that reflect typical duties of a Medical Assistant II. These tasks may include patient intake, recording vital signs, updating electronic medical records, assisting with a mock medical procedure, and responding to a series of patient inquiries and concerns.

### Objectives

- To evaluate the candidate's clinical aptitude and procedural knowledge.
- To assess the candidate’s competence with EMR software and accuracy in documentation.
- To observe the candidate’

# Create Team Interaction

In [79]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

Incorporating a team interaction component in the interview process allows us to assess a candidate's ability to integrate with Stanford Health Care - University Healthcare Alliance's culture and work harmoniously within our existing healthcare team. This stage of the assessment is designed to observe the candidate's interpersonal skills, collaborative spirit, and alignment with our foundational values of Teamwork and Connection.

### Rationale

The rationale for including team interaction in the assessment process is twofold:

- It offers a glimpse into how the candidate engages with potential colleagues, handles cooperative tasks, and contributes to group dynamics.
- It provides the team an opportunity to provide feedback on the candidate's fit - an essential factor, given the collaborative nature of clinical work.

### Discussion Points and Activities

During this stage, the candidate will engage in a few key activities:

- A team-based discussion on a relevant healthca

# Create Candidate Q&A

In [80]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

During the interview, candidates often have queries about the role, organization, and future opportunities. The following questions are anticipated from candidates applying for the Medical Assistant II position at Stanford Health Care - University Healthcare Alliance, with comprehensive answers to provide them with clarity and insight.

### Candidate Questions

1. **What opportunities for professional development does Stanford Health Care - University Healthcare Alliance offer to its medical assistants?**
   _We are committed to continual learning and offer various professional development programs, including onsite training sessions, workshops, and access to online courses to enhance your clinical skills and knowledge._

2. **How does the organization measure success in the Medical Assistant II role?**
   _Success is measured through a combination of patient satisfaction scores, adherence to clinical procedures and protocols, efficiency in managing tasks, and contrib

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [81]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('temp_files/playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [82]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [90]:
# Open the file and read the contents
with open('styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()

## Create some useful HTML snippets

In [84]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [91]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
  <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700" rel="stylesheet">
  <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700" rel="stylesheet">
  <style>
  {styles}
  </style>
</head>
<body>
    <section class="intro" id="intro">
    <div class="publication-date">Published on {publication_date} by the <a href="https://github.com/androb/deliverables/tree/main/interviewplaybook">Interview Playbook Generator</a>.</div>
    {html_introduction}
    </section>
    <section class='main' id="job_description">
    {html_job_description}
    </section>
    <section class='main' id="eval_criteria">
    {html_eval_criteria}
    </section>
    <section class='main' id="technical_assessment_1">
      {html_technical_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_short_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('temp_files/playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [92]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment,Comments
Technical: Clinical Procedures,,
Technical: Medical Knowledge,,
Technical: EMR Proficiency,,
Technical: Communication Skills,,
Technical: Multi-tasking and Organizational Skills,,
Behavioral: Patient-Centric Focus,,
Behavioral: Team Collaboration,,
Behavioral: Continuous Learning,,
"Behavioral: JEDI (Justice, Equality, Diversity, and Inclusion)",,
Behavioral: Initiative and Outcome-Driven Performance,,


# Create the Documents

Create the Word document

In [87]:
# Write HTML string to a file
with open('temp_files/playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s 'temp_files/playbook.html' -o temp_files/playbook.docx

# from google.colab import files
# files.download('playbook.docx')

Create the PDF

In [88]:
#!python3 --version
#!pango-view --version

#!brew install gobject-introspection
#!brew install gtk+3
#!brew install glade
#!brew install weasyprint

#!python3 -m venv venv
#!source venv/bin/activate
#%pip install cffi weasyprint
#!weasyprint --info

In [89]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('temp_files/playbook.html').write_pdf('temp_files/playbook.pdf')

# from google.colab import files
# files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')
